In [37]:
import pandas as pd

## 단어가 문서 내에 출현한 횟수를 구하는 함수
def get_term_frequency(document, word_dict = None):
    if word_dict is None:
        word_dict = {} #딕셔너리
    words = document.split()

    for w in words:
        word_dict[w] = 1 + (0 if word_dict.get(w) is None else word_dict[w])
    return pd.Series(word_dict).sort_values(ascending=False)

## 해당 단어가 출현한 문서의 수를 구하는 함수
def get_document_frequency(document):
    dicts = []
    vocab = set([])
    df = {} #딕셔너리

    for d in document:
        tf = get_term_frequency(d)
        dicts += [tf] #딕셔너리 저장
        vocab = vocab | set(tf.keys()) #단어들 저장
    
    for v in list(vocab):
        df[v] = 0
        for dict_d in dicts:
            if dict_d.get(v) is not None:
                df[v] += 1

    return pd.Series(df).sort_values(ascending=False)


In [8]:
## 다른 문서에 비해서 특정 문서에 많이 나온 단어를 찾는 함수
def get_tfidf(docs):
    vocab = {}
    tfs = []
    for d in docs:
        vocab = get_term_frequency(d, vocab)
        tfs += [get_term_frequency(d)]
    df = get_document_frequency(docs)

    from operator import itemgetter
    import numpy as np

    stats = []
    for word, freq in vocab.items():
        tfidfs = []
        for idx in range(len(docs)):
            if tfs[idx].get(word) is not None:
                tfidfs += [tfs[idx][word] * np.log(len(docs) / df[word])]
            else:
                tfidfs += [0]

        stats.append((word, freq, *tfidfs, max(tfidfs)))

    return pd.DataFrame(stats, columns=('word',
                                        'frequency',
                                        'news1',
                                        'news2',
                                        'news3',

                                        'max')).sort_values('max', ascending=False)

In [26]:
 ##기사가 저장되어 있는 csv 파일 읽어오기
 import os
 
 base_dir = 'C:\\Users\\gunhee\\Desktop\\NLP_practice'

 excel_file = 'Article_IT_202003.csv'

 excel_dir = os.path.join(base_dir, excel_file)


# read a excel file and make it as a DataFrame

 df_from_csv = pd.read_csv(excel_dir, # write your directory here 

                              header = None,

                              names = ['날짜', '분류', '언론사', '제목', '본문', 'url'],

                              na_values = 'NaN',  

                              nrows = 10, encoding = 'CP949')

In [29]:
print(df_from_csv.shape)
print(df_from_csv.ix[0][4])

(10, 6)
알싸한 기운속에서도 봄기운이 느껴지는 3월이 시작됐다. 지난 2월 한달은 코로나19 의 공포가 지배했다. 비교적 순항해왔던 코로나19 방역은 지난 2월20일 대구 경북 지역에서 신천지 신자들을 중심으로 집단적으로 확진자가 발생하면서 상황이 돌변했다. 정부는 위기경보 단계를 심각 단계로 올렸고 기업들은 가급적 재택근무에 돌입하는 등 비상체제로 전환했다. 통신사들 주요 IT기업들은 재택 근무 연장을 고려하고 있다. 이와함께 전세계 전역에서 코로나19 확진자가 빠르게 늘어나자 WHO 세계보건기구 는 각국에 강력한 선제적 조치를 취할 것을 당부했고 시장에선 펜데믹 대유행 우려로 요동쳤다. 미국 다우지수는 지난 27일 현지시간 사상 최대 폭의 기록적인 폭락을 기록했다. 국내 코스피지수도 지난 28일 2000선이 무너진채 장을 마감했다. 이번 주 금융시장도 이러한 답답한 흐름에서 큰 반전이 일어날 가능성은 높아 보이지 않는다. 방역에 총력전을 펼치고 있는 정부는 앞으로 1 2주가 고비가 될 것으로 보고 있다. 이런 가운데 코로나19 사태를 극복하고 공동체를 지키키위한 의료계의 헌신적인 노력 기업들의 후원 시민들의 응원도 크게 늘어나고 있다. 예년같으면 마케팅에 큰 활기를 뛰었을 3월이지만 코로나19 사태의 여파로 인파가 모이는 IT관련 행사는 없다. 삼성전자가 갤럭시S20 시리즈 사전예약을 당초 일정보다 일주일간 연장해 오는 3일까지 진행한다. 코로나19 확산으로 초반 판매 흥행이 어려웠기 때문이다. KT는 6일 5G 스트리밍 게임 오픈베타 서비스를 출시한다. 과학기술정보통신부는 ICT SW분야 인재 육성을 위해 ‘ICT ’멘토링 사업 프로젝트를 모집한다. 중국산 게임의 국내 진출이 주춤할 것으로 예상되는 가운데 크래프톤의 2020년 첫 신작 ‘테라 히어로’가 5일 출시된다. 코로나19 비상 통신업계 재택근무 계속될까 SK텔레콤과 KT가 코로나19 확산 여파로 전 직원 재택근무를 실시한다. 각각 이달 8일과 6일까지다. SK텔레콤은 당초 1일까지 재택근무를

In [33]:
## 읽어온 기사 형태소 단위로 분절하고 문자열로 변환하기
from konlpy.tag import Kkma
kkma = Kkma()
##형태소 추출
news = [' ' for i in range(3)]
for i in range(3):
    news[i]  = kkma.morphs(df_from_csv.ix[i][4])
    news[i] = ' '.join(news[i])


알싸 하 ㄴ 기운 속 에서 도 봄기운 이 느껴지 는 3 월 이 시작 되 었 다 . 지나 ㄴ 2 월 한 달 은 코로나 19 의 공포 가 지배 하 었 다 . 비교적 순항 하 어 오 았 더 ㄴ 코로나 19 방역 은 지나 ㄴ 2 월 20 일 대구 경북 지역 에서 신천지 신자 들 을 중심 으로 집단적 으로 확 진자 가 발생 하 면서 상황 이 돌변 하 었 다 . 정부 는 위기 경보 단계 를 심각 단계 로 올리 었 고 기업 들 은 가급적 재택근무 에 돌입 하 는 등 비상 체제 로 전환 하 었 다 . 통신사 들 주요 IT 기업 들 은 재택 근무 연장 을 고려 하 고 있 다 . 이 와 함께 전 세계 전역 에서 코로나 19 확 진자 가 빠르 게 늘어나 자 WHO 세계 보건 기구 늘 ㄴ 각국 에 강력 하 ㄴ 선제 적 조치 를 취하 ㄹ 것 을 당부 하 었 고 시장 에서 는 펜 데 믹 대유행 우려 로 요동치 었 다 . 미국 다우 지수 는 지나 ㄴ 27 일 현지 시간 사상 최대 폭 의 기록 적인 폭락 을 기록하 었 다 . 국내 코스 피 지수 도 지나 ㄴ 28 일 2000 선 이 무너지 ㄴ 채 장 을 마감 하 었 다 . 이 번 주 금융 시장 도 이러하 ㄴ 답답 하 ㄴ 흐름 에서 크 ㄴ 반전 이 일어나 ㄹ 가능성 은 높 아 보이 지 않 는 다 . 방역 에 총력전 을 펼치 고 있 는 정부 는 앞 으로 1 2 주 가 고비 가 되 ㄹ 것 으로 보 고 있 다 . 이런 가운데 코로나 19 사태 를 극복 하 고 공동체 를 지 키 키위 하 ㄴ 의료계 의 헌신적 이 ㄴ 노력 기업 들 의 후원 시민 들 의 응원 도 크 게 늘어나 고 있 다 . 예년 같 으면 마케팅 에 크 ㄴ 활기 를 뛰 었 을 3 월 이 지만 코로나 19 사태 의 여파 로 인파 가 모이 는 IT 관련 행사 는 없 다 . 삼성전자 가 개 ㄹ 럭 시 S 20 시리즈 사전 예약 을 당초 일정 보다 일주일간 연장 하 어 오 는 3 일 까지 진행하 ㄴ다 . 코로나 19 확산 으로 초반 판매 흥행 이 어렵 었 기 때문 이 다 . KT 는 6 일 5

In [40]:
#TF-IDF 구하기
get_tfidf(news)

,word,frequency,news1,news2,news3,max
28,게임,13,14.281960,0.0,0.000000,14.281960
32,재택근무,11,12.084735,0.0,0.000000,12.084735
39,센터,9,0.000000,0.0,9.887511,9.887511
13,의,27,9.325697,0.0,1.621860,9.325697
47,연장,8,8.788898,0.0,0.000000,8.788898
...,...,...,...,...,...,...
33,아,11,0.000000,0.0,0.000000,0.000000
31,확,11,0.000000,0.0,0.000000,0.000000
30,1,12,0.000000,0.0,0.000000,0.000000
25,에서,15,0.000000,0.0,0.000000,0.000000
